# Thinking1：XGBoost与GBDT的区别是什么？
#### 1、传统GBDT在优化时只用到一阶导数信息；XGBoost的损失函数是用泰勒展开式展开的，用到了一阶导和二阶导，可以加快优化速度。
#### 2、传统GBDT以CART作为基分类器；XGBoost不仅支持CART作为基分类器，还支持线性分类器，在使用线性分类器的时候可以使用L1，L2正则化。
#### 3、xgboost在代价函数里加入了正则化项，用于控制模型的复杂度。正则项降低了模型的variance，使学习出来的模型更加简单，防止过拟合，泛化性能好.
# Thinking2：举一个你之前做过的预测例子
#### answer：上周，我用libFM对movielens数据集进行了电影评分预测，发现预测值和真实值的误差的波动性较大，经过老师的讲解后，推测原因之一应该是测试集和训练集的划分方法不太科学，同一userid要么只在训练集出现，要么只在训练集出现，导致样本随机性不够。接下来会尝试新的切分方法并与之前的进行比较。另外，在libFM的使用过程中，注意到SGD算法必须要指定learn_rate，否则程序会报错；还发现各参数取值对模型的影响很大，应根据经验和实际预测结果来调整参数（如学习率和迭代次数），以得到更科学的预测值。
# Thinking3：你思考，在你的工作中，需要构建哪些特征（比如用户画像，item特征...），这些特征都包括哪些维度
#### answer：本人从事的是电商行业，平时在进行用户画像分析时，主要考虑的特征有：顾客年龄、性别、地域、兴趣、粉丝与店铺亲密度等，然后针对不同特点的人群进行千人千面的页面展示与营销。

In [6]:
#Action 1
import pandas as pd
import numpy as np

#数据加载
df = pd.read_csv('./voice.csv')

print('样本个数：{}'.format(df.shape[0]))
print('男性样本个数：{}'.format(df[df.label=='male'].shape[0]))
print('女性样本个数：{}'.format(df[df.label=='female'].shape[0]))

#分割特征和Target(最后一列为Target)
X = df.iloc[:, :-1]

#使用标签编码
from sklearn.preprocessing import LabelEncoder
y = df.iloc[:, -1]
gender_encoder = LabelEncoder()
y = gender_encoder.fit_transform(y)
#print(y)

#将特征X进行数据规范化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
#print(X)

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2)

from sklearn.svm import SVC
from sklearn import metrics

#用XGBoost进行男女声音识别
import xgboost as xgb
#模型参数配置
param = {'boosting_type':'gbdt',
                         'objective' : 'binary:logistic', #任务目标
                         'eval_metric' : 'auc', #评估指标
                         'eta' : 0.01, #学习率
                         'max_depth' : 15, #树最大深度
                         'colsample_bytree':0.8, #设置在每次迭代中使用特征的比例
                         'subsample': 0.9, #样本采样比例
                         'subsample_freq': 8, #bagging的次数
                         'alpha': 0.6, #L1正则
                         'lambda': 0, #L2正则
        }
#训练模型，进行预测
train_data = xgb.DMatrix(X_train, label=y_train)
test_data = xgb.DMatrix(X_test, label=y_test)
model = xgb.train(param, train_data, evals=[(train_data, 'train'), (test_data, 'test')], num_boost_round = 1500, early_stopping_rounds=200, verbose_eval=25)
y_pred = model.predict(test_data)
y_pred = [1 if x>=0.5 else 0 for x in y_pred]
print('XGBoost 预测结果：',y_pred)
print('XGBoost 准确率：',metrics.accuracy_score(y_test, y_pred))


样本个数：3168
男性样本个数：1584
女性样本个数：1584
[21:58:45] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { boosting_type, subsample_freq } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.99169	test-auc:0.98890
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 200 rounds.
[25]	train-auc:0.99947	test-auc:0.99577
[50]	train-auc:0.99946	test-auc:0.99577
[75]	train-auc:0.99955	test-auc:0.99714
[100]	train-auc:0.99969	test-auc:0.99756
[125]	train-auc:0.99975	test-auc:0.99778
[150]	train-auc:0.99980	test-auc:0.99783
[175]	train-auc:0.99985	test-auc:0.99781
[200]	train-auc:0.99987	test-auc:0.99785
[225]	train-auc:0.99989	test-auc:0.99788
[250]	train-